# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "output_data/cities.csv"
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,City_ID,City,Country,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Lng,Lat
0,0,Hermanus,ZA,1651350334,14.96,79,1.34,86,19.2345,-34.4187
1,1,Impfondo,CG,1651350335,24.32,12,0.80,68,18.0667,1.6381
2,2,Rikitea,PF,1651350335,24.40,51,5.43,76,-134.9692,-23.1203
3,3,Port Alfred,ZA,1651350336,13.99,1,0.89,83,26.8910,-33.5906
4,4,Emba,KZ,1651350336,13.82,100,7.65,66,58.1442,48.8264


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
cities = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

In [5]:
# Add Heatmap layer to map.
fig = gmaps.figure()

# Add the layer to the map
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, point_radius=5)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#  * A max temperature lower than 80 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.

optimized_weather = cities_df.loc[(cities_df["Max Temp"]<27) & (cities_df["Max Temp"]>21)\
                                 & (cities_df["Wind Speed"]<10)\
                                 & (cities_df["Cloudiness"]==0)
                                 ].dropna()
optimized_weather.describe()



,City_ID,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Lng,Lat
count,12.000000,1.200000e+01,12.000000,12.0,12.000000,12.000000,12.000000,12.000000
mean,370.333333,1.651350e+09,23.100000,0.0,5.323333,44.500000,-65.862342,16.888817
std,176.959334,1.434722e+02,1.786057,0.0,2.386570,24.897974,60.898667,24.257580
min,22.000000,1.651350e+09,21.120000,0.0,1.970000,9.000000,-158.154700,-26.366700
25%,292.750000,1.651350e+09,22.180000,0.0,3.550000,28.500000,-90.454650,7.595275
50%,434.000000,1.651351e+09,22.410000,0.0,4.550000,43.000000,-73.133350,23.039150
75%,496.500000,1.651351e+09,23.232500,0.0,7.720000,61.000000,-15.458075,37.220550
max,546.000000,1.651351e+09,26.970000,0.0,9.770000,88.000000,46.721900,39.183400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame(optimized_weather)
hotel_df

,City_ID,City,Country,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Lng,Lat
22,22,Dakar,SN,1651350239,22.49,0,3.09,88,-17.4441,14.6937
87,87,Tiznit Province,MA,1651350356,21.12,0,1.97,36,-9.5000,29.5833
202,202,Mayo,US,1651350267,22.23,0,4.47,32,-76.5119,38.8876
323,323,Pisco,PE,1651350308,22.03,0,7.72,64,-76.2167,-13.7000
393,393,Nānākuli,US,1651350540,26.97,0,3.60,59,-158.1547,21.3906
401,401,Farmington,US,1651350500,22.33,0,4.63,10,-108.2187,36.7281
467,467,Riyadh,SA,1651350409,24.08,0,3.40,24,46.7219,24.6877
469,469,Hamilton,US,1651350579,22.95,0,5.66,72,-84.5333,39.1834
496,496,Calama,CL,1651350591,22.27,0,9.77,9,-68.9333,-22.4667
498,498,Cascais,PT,1651350592,21.64,0,7.72,50,-9.4215,38.6979


In [8]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [9]:
# * Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.
# Set parameters to search for hotels with 5000 meters.
params = {"type" : "lodging ",
             "radius" : 5000,
             "key": g_key

             }
# search
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["cities"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    address = requests.get(url, params=params)
    
    # convert to json
    address = address.json()
    
    # Store the first Hotel result into the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Not found ... skipping.")

hotel_df
# Hit the Google Places API for each city's coordinates.
# Plot markers on top of the heatmap.

Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.
Not found ... skipping.


,City_ID,City,Country,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Lng,Lat,Hotel Name
22,22,Dakar,SN,1651350239,22.49,0,3.09,88,-17.4441,14.6937,
87,87,Tiznit Province,MA,1651350356,21.12,0,1.97,36,-9.5000,29.5833,
202,202,Mayo,US,1651350267,22.23,0,4.47,32,-76.5119,38.8876,
323,323,Pisco,PE,1651350308,22.03,0,7.72,64,-76.2167,-13.7000,
393,393,Nānākuli,US,1651350540,26.97,0,3.60,59,-158.1547,21.3906,
401,401,Farmington,US,1651350500,22.33,0,4.63,10,-108.2187,36.7281,
467,467,Riyadh,SA,1651350409,24.08,0,3.40,24,46.7219,24.6877,
469,469,Hamilton,US,1651350579,22.95,0,5.66,72,-84.5333,39.1834,
496,496,Calama,CL,1651350591,22.27,0,9.77,9,-68.9333,-22.4667,
498,498,Cascais,PT,1651350592,21.64,0,7.72,50,-9.4215,38.6979,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(cities, info_box_content=hotel_info)
# Add the layer to the map
#fig.add_layer(markers)
#fig

IndexError: list index out of range

In [ ]:
# Add marker layer ontop of heat map


# Display figure
